In [147]:
import os
import pandas as pd

In [148]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

In [149]:
files = list(set(os.listdir('backend/data/odds')).difference({'odds.csv', 'odds.ipynb', 'nfl odds 2022-23.xlsx', 'nfl odds 2021-22.xlsx'}))
files

['nfl odds 2020-21.xlsx',
 'nfl odds 2019-20.xlsx',
 'nfl odds 2017-18.xlsx',
 'nfl odds 2016-17.xlsx',
 'nfl odds 2015-16.xlsx',
 'nfl odds 2010-11.xlsx',
 'nfl odds 2014-15.xlsx',
 'nfl odds 2013-14.xlsx',
 'nfl odds 2018-19.xlsx',
 'nfl odds 2011-12.xlsx',
 'nfl odds 2012-13.xlsx']

In [150]:
def spreads(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col1
    else:
        return col2

In [151]:
def totals(col1, col2):
    if col1 == "pk":
        col1 = 0
    if col2 == "pk":
        col2 = 0
    if col1 < col2:
        return col2
    else:
        return col1

In [152]:
def odds_season(filename):
    df = pd.read_excel(f'backend/data/odds/{filename}')
    df.columns = [col.lower() for col in df.columns]

    df["year"] = filename.split("-")[0][-4:]

    home_df = df[df["vh"] == "H"].reset_index()[["team", "ml", "close", "year", "week"]]
    home_df.rename(columns={"team": "home"}, inplace=True)

    away_df = df[df["vh"] == "V"].reset_index()[["team", "ml", "close", "year", "week"]]
    away_df.rename(columns={"team": "away"}, inplace=True)

    df = home_df.join(away_df, lsuffix="_h", rsuffix="_a")

    df['spread'] = df.apply(lambda x: spreads(x.close_h, x.close_a), axis=1)
    df['total'] = df.apply(lambda x: totals(x.close_h, x.close_a), axis=1)
    df.drop(["close_h", "close_a"], axis=1, inplace=True)

    df.drop(["week_a", "year_a"], axis=1, inplace=True)
    df.rename(columns={"year_h": "year", "week_h": "week"}, inplace=True)

    teams_dict = {
        'Philadelphia': 'philadelphia-eagles', 'St.Louis': 'st-louis-rams',
        'TampaBay': 'tampa-bay-buccaneers', 'NYGiants': 'new-york-giants', 'GreenBay': 'green-bay-packers',
        'Chicago': 'chicago-bears', 'NewEngland': 'new-england-patriots', 'Pittsburgh': 'pittsburgh-steelers',
        'Houston': 'houston-texans', 'Denver': 'denver-broncos', 'SanFrancisco': 'san-francisco-49ers',
        'Minnesota': 'minnesota-vikings', 'Washington': 'washington-redskins',
        'Jacksonville': 'jacksonville-jaguars', 'Tennessee': 'tennessee-titans',
        'Carolina': 'carolina-panthers', 'KansasCity': 'kansas-city-chiefs', 'Miami': 'miami-dolphins',
        'Atlanta': 'atlanta-falcons', 'NewOrleans': 'new-orleans-saints', 'Baltimore': 'baltimore-ravens',
        'Seattle': 'seattle-seahawks', 'SanDiego': 'san-diego-chargers', 'Dallas': 'dallas-cowboys',
        'Cincinnati': 'cincinnati-bengals', 'NYJets': 'new-york-jets', 'Detroit': 'detroit-lions',
        'Arizona': 'arizona-cardinals', 'Oakland': 'oakland-raiders', 'Indianapolis': 'indianapolis-colts',
        'Buffalo': 'buffalo-bills', 'Cleveland': 'cleveland-browns', 'LARams': 'los-angeles-rams',
        'LAChargers': 'los-angeles-chargers', 'washington': 'washington-football-team',
        'LasVegas': 'las-vegas-raiders'
    }

    df["home"] = df["home"].apply(lambda x: teams_dict[x])
    df["away"] = df["away"].apply(lambda x: teams_dict[x])

    return df

In [154]:
odds = pd.DataFrame()
for file in files:
    odds = odds.append(odds_season(file))
odds = pd.concat(
    [
        odds, 
        pd.read_excel('backend\\data\\odds\\nfl odds 2021-22.xlsx'),
        pd.read_excel('backend\\data\\odds\\nfl odds 2022-23.xlsx')
    ],
    axis=0
)
odds.tail(20)

,home,ml_h,year,week,away,ml_a,spread,total
281,kansas-city-chiefs,-130,2021,20,buffalo-bills,110,2.5,54.0
282,kansas-city-chiefs,-350,2021,21,cincinnati-bengals,270,7.0,54.5
283,los-angeles-rams,-180,2021,21,san-francisco-49ers,155,3.5,45.5
284,cincinnati-bengals,160,2021,22,los-angeles-rams,-190,4.0,48.5
0,los-angeles-rams,120,2022,1,buffalo-bills,-134,2.5,51.5
1,atlanta-falcons,190,2022,1,new-orleans-saints,-205,5.0,41.0
2,chicago-bears,235,2022,1,san-francisco-49ers,-275,6.5,42.0
3,cincinnati-bengals,-270,2022,1,pittsburgh-steelers,245,6.5,44.5
4,detroit-lions,180,2022,1,philadelphia-eagles,-180,3.5,46.5
5,miami-dolphins,-140,2022,1,new-england-patriots,123,3.0,44.5


In [155]:
odds.to_csv('backend\\data\\odds\\odds.csv', index=False)